In [ ]:
import aiohttp
import asyncio
import json
import nest_asyncio
nest_asyncio.apply()


class Generator():

    def __init__(self, model: str, openrouter_key: str, providers: list, temperature: float = 1.0):
        """
        Initializes the generator with the specified model, OpenRouter key, temperature, and providers.
        Args:
            model (str): The name or path of the pre-trained model to use.
            openrouter_key (str): The API key for OpenRouter.
            temperature (float, optional): The temperature to use for sampling. Defaults to 0.9.
            providers (list, optional): A list of providers to use. Defaults to None.
        """
        self.model = model
        self.openrouter_key = openrouter_key
        self.temperature = temperature
        self.providers = providers if providers else []
    
    async def generate(self, prompt: str) -> str:
        """
        Asynchronously generates a response based on the given prompt using the specified model.
        """
        out = None
        async with aiohttp.ClientSession() as session:
            while out is None:
                payload = json.dumps({
                    'model': self.model,
                    'providers': {"order": self.providers},
                    'temperature': self.temperature,
                    'messages': [
                        {
                            'role': 'user',
                            'content': prompt,
                        }
                    ],
                })

                headers = {
                    'Authorization': f'Bearer {self.openrouter_key}'
                }

                try:
                    url = 'https://openrouter.ai/api/v1/chat/completions'
                    async with session.post(url, headers=headers, data=payload) as response:
                        if response.status == 200:
                            try:
                                response_json = await response.json()
                                tryout = response_json['choices'][0]['message']['content']
                                tryout = tryout.strip().strip("b'").strip()
                                if tryout.startswith('An error occurred:') or tryout == '':
                                    print('Error? Retrying...')
                                    print(tryout)
                                    await asyncio.sleep(0.5)
                                    continue
                                out = tryout
                            except:
                                out = str(await response.text())
                        else:
                            print(response.status)
                            await asyncio.sleep(0.5)
                except:
                    print('Error in sending post request? Retrying...')
                    await asyncio.sleep(0.5)
                    continue

        return out

In [ ]:
import os
import json
import asyncio
import aiohttp
from tqdm import tqdm
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

class Generator:
    def __init__(self, model: str, openrouter_key: str, providers: list, temperature: float = 1.0):
        """
        Initialize the generator with the model, OpenRouter key, list of providers, and temperature.
        """
        self.model = model
        self.openrouter_key = openrouter_key
        self.temperature = temperature
        self.providers = providers if providers else []
    
    async def generate(self, prompt: str) -> str:
        """
        Asynchronously generate a response based on the given prompt.
        """
        out = None
        async with aiohttp.ClientSession() as session:
            while out is None:
                payload = json.dumps({
                    'model': self.model,
                    'providers': {"order": self.providers},
                    'temperature': self.temperature,
                    'messages': [
                        {
                            'role': 'user',
                            'content': prompt,
                        }
                    ],
                })

                headers = {
                    'Authorization': f'Bearer {self.openrouter_key}'
                }

                try:
                    url = 'https://openrouter.ai/api/v1/chat/completions'
                    async with session.post(url, headers=headers, data=payload) as response:
                        if response.status == 200:
                            try:
                                response_json = await response.json()
                                tryout = response_json['choices'][0]['message']['content']
                                tryout = tryout.strip().strip("b'").strip()
                                # If there is an error in the response, retry
                                if tryout.startswith('An error occurred:') or tryout == '':
                                    print('Error in response, retrying...')
                                    await asyncio.sleep(0.5)
                                    continue
                                out = tryout
                            except Exception as e:
                                out = str(await response.text())
                        else:
                            print("HTTP status:", response.status)
                            await asyncio.sleep(0.5)
                except Exception as e:
                    print('Error sending the request, retrying...')
                    await asyncio.sleep(0.5)
                    continue

        return out


async def evaluate_candidate(generator: Generator, response_text: str, baseline_response: str, role: str) -> str:
    """
    Constructs the prompt and repeats the request until a valid response containing [Yes] or [No] is obtained.
    """
    instruction = f"""Quick {role} Check

Text to Evaluate:
'{response_text}'

Baseline Reference:
'{baseline_response}'

We are testing whether the LLM has been successfully steered from its baseline output to adopt a {role} perspective. Your task is to determine if the response contains content that is relevant to the {role} domain. Note that you are not assessing the quality of the response—only its relation to the {role} concept.

Please keep in mind:
- The text may be repetitive or somewhat incoherent.
- If the response closely mirrors the baseline without introducing any distinct {role}-related elements, it should be considered as not aligned with the {role} role.

Evaluate this step by step and answer the following question:
Is the model being steered toward the {role} role?

Reply with your final answer in the format: [Yes] or [No].
"""
    # Retry loop until a valid response is received
    while True:
        completion = await generator.generate(instruction)
        if "[Yes]" in completion or "[No]" in completion:
            return completion
        print("Invalid response, retrying...")


async def process_candidate_file(file_path: str, baseline_response: str, role: str, generator: Generator):
    print(f"Processing file: {file_path}")
    try:
        with open(file_path, "r") as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return

    # Assuming the file contains a list of dictionaries (completions)
    modified = False
    for item in data:
        # If the fields "passed" or "completion" are "Error", proceed with evaluation
        if item.get("passed") == "Error" or item.get("completion") == "Error":
            response_text = item.get("response", "")
            evaluation = await evaluate_candidate(generator, response_text, baseline_response, role)
            # Update fields based on the evaluation response
            if "[Yes]" in evaluation:
                item["passed"] = "Yes"
            elif "[No]" in evaluation:
                item["passed"] = "No"
            item["completion"] = evaluation
            modified = True
        else:
            # If there are no errors, the file has already been processed
            print(f"The file {file_path} is already processed, skipping.")

    # Overwrite the file if modifications were made
    if modified:
        with open(file_path, "w") as f:
            json.dump(data, f, indent=4)
        print(f"File {file_path} updated.")
    else:
        print(f"No changes for file {file_path}.")


async def process_role(model_path: str, role: str, generator: Generator):
    print(f"Processing role: {role} for model: {model_path}")
    role_path = os.path.join(model_path, role)
    test_direction_path = os.path.join(role_path, "test_direction")
    baseline_file = os.path.join(test_direction_path, "baseline_completions.json")
    
    if not os.path.exists(baseline_file):
        print(f"Baseline file not found for role '{role}' in {model_path}.")
        return

    # Load the baseline file
    try:
        with open(baseline_file, "r") as f:
            baseline_data = json.load(f)
    except Exception as e:
        print(f"Error loading baseline file {baseline_file}: {e}")
        return

    # Use the first element as baseline reference; adjust as needed
    if len(baseline_data) == 0:
        print(f"Baseline file is empty for role '{role}' in {model_path}.")
        return
    baseline_response = baseline_data[0].get("response", "")

    # Path to folder 3.0
    dir_3_0 = os.path.join(test_direction_path, "3.0")
    if not os.path.isdir(dir_3_0):
        print(f"Folder '3.0' not found for role '{role}' in {model_path}.")
        return

    # Find all JSON files in folder 3.0
    candidate_files = [os.path.join(dir_3_0, f) for f in os.listdir(dir_3_0) if f.endswith(".json")]
    tasks = []
    for file_path in candidate_files:
        tasks.append(process_candidate_file(file_path, baseline_response, role, generator))
    if tasks:
        await asyncio.gather(*tasks)
    else:
        print(f"No candidate file found in {dir_3_0} for role '{role}'.")


async def process_model(model_path: str, generator: Generator):
    print(f"\nProcessing model: {model_path}")
    # Get all subdirectories of the model (considered roles)
    roles = [d for d in os.listdir(model_path) if os.path.isdir(os.path.join(model_path, d))]
    if not roles:
        print(f"No roles found in {model_path}.")
        return

    # Process roles sequentially
    for role in roles:
        await process_role(model_path, role, generator)
    print(f"Processing model {model_path} completed.")


async def main(models_root: str):
    # Generator configuration: replace with the correct values
    model_test = "anthropic/claude-3.5-haiku"  # or the model of your choice
    openrouter_key = os.getenv("OPENROUTER_KEY")
    if not openrouter_key:
        print("Error: OPENROUTER_KEY has not been set in the .env file")
        return

    providers_test = ["Amazon Bedrock", "Anthropic"]
    temperature_test = 0

    generator = Generator(
        model=model_test, 
        openrouter_key=openrouter_key, 
        providers=providers_test, 
        temperature=temperature_test
    )

    # Find all model directories in the models_root
    model_dirs = [os.path.join(models_root, d) for d in os.listdir(models_root)
                  if os.path.isdir(os.path.join(models_root, d))]
    
    # Process models sequentially with a loading bar
    for model_path in tqdm(model_dirs, desc="Processing models"):
        await process_model(model_path, generator)


In [ ]:
models_root = r"C:\Users\user\Desktop\temp\rolevectors_results"  


await main(models_root)
